# Pypam

## 0. Setup (install)

First we need to install all the packages which we need to exectue this notebook. You don't need to do this if you're using mybinder

In [ ]:
!pip install pvlib # This one is optional for the summary plot! 
!pip install git+https://github.com/lifewatch/pypam@feature/hmb_to_octaves # install pypam's latest version (from github)
!pip install minio # Install package to download the process

## 1. Download the data

We will download some processed HMB data stored in the cloud to give some examples of how can it be used. 
These data will be downloaded in this jupyterlab space, and you will be able to find them under the folder you specify here below, organized by station. 
Please change this line depending on where you want to store the data. 

In [1]:
local_path = './data'

We will start defining a function to download data which can be then reused for different stations (then we don't need to write as much!)

We first start importing the packages we need for this part of the notebook

In [2]:
# Import the necessary packages
import minio
import os
import pathlib
from functools import partial
import xarray
import dask
import pypam

In [3]:
def open_dataset_cloud(object_name, data_folder, bucket, client, drop_variables):
    object_data = client.get_object(bucket, object_name)
    download_path = data_folder + '/' + object_name.replace('/', '_')
    with open(str(download_path), 'wb') as file_data:
        for data in object_data:
            file_data.write(data)
    ds = xarray.open_dataset(download_path, engine='netcdf4', drop_variables=drop_variables)
    ds.attrs['download_path'] = download_path
    ds.close()
    return ds

In [4]:
def preprocess(ds, freq_band, freq_coord, time_resample, data_vars, datetime_coord):
    if freq_band is not None:
        ds = pypam.utils.select_frequency_range(ds, freq_band[0], freq_band[1], freq_coord=freq_coord)
    if time_resample is not None:
        ds = ds.resample({datetime_coord: time_resample}).median()

    if data_vars is not None:
        ds = ds[data_vars]

    return ds

In [5]:
def open_mfdataset_cloud(
        paths,
        preprocess,
        data_vars,
        bucket,
        client,
        data_folder,
        **kwargs,
):
    open_kwargs = dict(bucket=bucket, client=client, data_folder=data_folder, **kwargs)

    # wrap the open_dataset, getattr, and preprocess with delayed
    open_ = dask.delayed(open_dataset_cloud)
    getattr_ = dask.delayed(getattr)
    if preprocess is not None:
        preprocess = dask.delayed(preprocess)

    datasets = [open_(p, **open_kwargs) for p in paths]
    closers = [getattr_(ds, "_close") for ds in datasets]
    remover_ = dask.delayed(os.remove)
    if preprocess is not None:
        datasets = [preprocess(ds) for ds in datasets]

    removers = [remover_(ds.attrs['download_path']) for ds in datasets]
    
    datasets, closers = dask.compute(datasets, closers, removers)

    # Combine all datasets, closing them in case of a ValueError
    combined = xarray.combine_by_coords(
        datasets,
        compat="no_conflicts",
        data_vars=data_vars,
        coords="different",
        join="outer",
        combine_attrs="override",
    )

    combined.set_close(partial(xarray._multi_file_closer, closers))

    return combined

In [6]:
def load_data_station(data_folder, client_obj, bucket_str, prefix_str, year, data_vars, freq_coord,
                      datetime_coord, freq_band, time_resample):
    files_to_load = []
    objects = list(client_obj.list_objects(bucket_str, prefix=prefix_str))
    for i, obj in enumerate(objects):
        object_name = obj.object_name
        path_name = pathlib.Path(object_name).name
        if (not path_name.startswith('.')) and path_name.endswith('.nc'):
            if str(year) in path_name:
                files_to_load.append(object_name)
            else:
                print('Ignored: ' + path_name)
    partial_func = partial(preprocess, data_vars=data_vars, datetime_coord=datetime_coord,
                           freq_band=freq_band, time_resample=time_resample, freq_coord=freq_coord)
    ds_tot = open_mfdataset_cloud(files_to_load,
                                  preprocess=partial_func,
                                  data_vars=data_vars,
                                  bucket=bucket,
                                  client=client,
                                  data_folder=data_folder,
                                  drop_variables=['psd_image_colormap',
                                                  'psd_image',
                                                  'percentile_image',
                                                  'percentile_image_colormap'])
    return ds_tot

In [7]:
from tqdm import tqdm
def load_data_station_slow(data_folder, client_obj, bucket_str, prefix_str, year, data_vars, freq_coord,
                      datetime_coord, freq_band, time_resample):
    objects = list(client_obj.list_objects(bucket_str, prefix=prefix_str))
    ds_total = xarray.Dataset()

    list_to_remove = []
    for i, obj in enumerate(tqdm(objects)):
        object_name = obj.object_name
        path_name = pathlib.Path(object_name).name

        if (not path_name.startswith('.')) and path_name.endswith('.nc'):
            if str(year) in path_name:
                object_data = client.get_object(bucket, object_name)
                download_path = data_folder + '/' + object_name.replace('/', '_')
                with open(str(download_path), 'wb') as file_data:
                    for data in object_data:
                        file_data.write(data)
                list_to_remove.append(download_path)
            else:
                print('Ignored: ' + path_name)
            if i % 3 == 0: 
                partial_func = partial(preprocess, data_vars=data_vars, datetime_coord=datetime_coord,
                                           freq_band=freq_band, time_resample=time_resample, freq_coord=freq_coord)
                ds_10 = xarray.open_mfdataset(pathlib.Path(data_folder).glob('*.nc'), parallel=True, preprocess=partial_func, data_vars=data_vars)
                ds_total = xarray.merge([ds_total, ds_10])
                for d in list_to_remove: 
                    os.remove(d)
                list_to_remove = []
                

    return ds_total

Then we will download the data for 2 stations. 
For this workshop we have chosen MARS and NRS11, but feel free to change the station names here if you wish to play around with other stations from the same providers

Let's start with MARS. We first need to describe where the data are located

In [ ]:
# Set up the download for MARS data
client = minio.Minio( "s3.us-west-2.amazonaws.com", secure=False)

bucket = 'pacific-sound-spectra'
prefix = '2021/'
station = 'MARS'

ds_mars = load_data_station_slow(data_folder=local_path, client_obj=client, bucket_str=bucket,
                  prefix_str=prefix, year=2021, data_vars=['psd'], freq_coord='frequency',
                  datetime_coord='time', freq_band=[0, 12000], time_resample='1h')


  0%|          | 0/711 [00:00<?, ?it/s]

Then we move to the NRS11 data. We repreat the same process just with different parameters.
These are the data we can find at [sanctsound](https://console.cloud.google.com/storage/browser/noaa-passive-bioacoustic/sanctsound?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&prefix=&forceOnObjectsSortingFiltering=false), daily millidecade bands. 

In [ ]:
# Set up the download for NRS11 data 
client = minio.Minio('storage.googleapis.com')
bucket = 'noaa-passive-bioacoustic'
station = 'NRS11'
prefix = 'soundcoop/%s/' % station
download_data_station(station, client, bucket, prefix, local_path, year=2021)

## 2. Let's look at the data

We will first import all the packes we need to execut this part of the notebook

In [ ]:
# Import pypam modules 
import pypam.utils
import pypam.plots

In [ ]:
# Import other tools
import pathlib
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

Let's have a look at one of these netcdf files! You can click on the document sign or the database sign once the dataset is shown here below to get extra information. 

In [ ]:
# Select one of the daily files
# you can browse through the daily netcdf files and select nay of the files we just downloaded, and copy the path here
ds = xr.open_dataset(local_path + '/NRS11/NRS11_H5R6.1.5000_20191108_DAILY_MILLIDEC_MinRes_v2.nc')

In [ ]:
# Show the file information
ds

We can then plot the long-term spectrogram of this day, selecting only the frequencies that we are interested in

In [ ]:
# Select the minimum and the maximum frequency
min_freq = 10
max_freq = 10000

# Plot the LTSA of one day
fig, ax = plt.subplots(figsize=(10, 5))
ds = pypam.utils.select_frequency_range(ds, min_freq=min_freq, max_freq=max_freq, freq_coord='frequency')
pypam.plots.plot_ltsa(ds=ds, data_var='psd', log=True, save_path=None, ax=ax, show=False, freq_coord='frequency',
                      time_coord='time')

We can also apply the quality flag to plot these data (only for data where it is available, not for MARS): 

In [ ]:
# Plot the LTSA of one day with the mask
minimum_quality = 2 # you can change this value depending on what you're interested in seeing. Check the ds quality_flag properties to see what each value (1 to 4) means
ds_masked = ds.copy() 
ds_masked['psd'] = ds_masked.psd.where(ds_masked.quality_flag < minimum_quality)
fig, ax = plt.subplots(figsize=(10, 5))
pypam.plots.plot_ltsa(ds=ds_masked, data_var='psd', log=True, save_path=None, ax=ax, show=False, freq_coord='frequency',
                      time_coord='time')
ax.set_facecolor('black') # -> this allows us to see in black where the mask has been applied 

We can also specify the location and then the day/night patter will be shown. But then we need to install the pvblib package (already done at the beginning of the notebook)

First we will get where the location is from the metadata in the netdf file. We can find where is the property of that name when we were looking into the data some cells above.

In [ ]:
location_str = ds.geospatial_bounds.replace('(', '').replace(')', '').split(' ')[1:]
location = [float(l) for l in location_str]

In [ ]:
# Lon, Lat
min_psd = 35 # in db
max_psd = 120 # in db
h = 1 # in db
percentiles = [1, 5, 10, 50, 90, 99]
pypam.plots.plot_summary_dataset(ds, percentiles, data_var='psd', time_coord='time', freq_coord='frequency',
                                 min_val=min_psd, max_val=max_psd, show=True, log=True, save_path=None,
                                 location=location)

## 3. Long-term plotting

In this section we will show some tools for resampling data and long-term plotting. For this, we can load all the computed daily millidecade bands in one xarray dataset for each station, and we will save the output as one file (a full year per station!). 
For this we will load the data using  the join_all_ds_output_deployment of pypam, which uses Dask parallel computing for fast access. 

We will load only a part of the data (in frequency range), because the MARS data has spectrum values up to 100 kHz, and we want to keep it light. 

In [ ]:
RESAMPLE_RESOLUTION = '1h' # we could also set it to 1d for daily resolution for example
min_freq = 10
max_freq = 10000

In [ ]:
# Only get data from 2021
def load_data_from_station_year(station, year, data_vars):
    deployment_path = pathlib.Path(local_path).joinpath(station)
    print('loading station %s...' % station)
    aggregated_ds = pypam.utils.join_all_ds_output_deployment(deployment_path, data_vars=data_vars, datetime_coord='time',
                                                              join_only_if_contains='_%s' % year, load=True,
                                                              parallel=False, freq_band=[min_freq, max_freq],
                                                              freq_coord='frequency',
                                                              drop_variables=['psd_image_colormap',
                                                                              'psd_image',
                                                                              'percentile_image',
                                                                              'percentile_image_colormap'])
    return aggregated_ds # this assigns an xarray dataset

<div class="alert alert-block alert-info">
⚠️ Be patient, loading two full years can take a while...
</div>

In [ ]:
# Load the data for each station
# Quality flag is only available for NRS11, not for MARS. We can decide to ignore it or load it by removing it from the data_vars list 
ds_collection = {}
ds_collection['MARS'] = load_data_from_station_year('MARS', 2021, data_vars=['psd'])
ds_collection['NRS11'] = load_data_from_station_year('NRS11', 2021, data_vars=['psd', 'quality_flag'])

### Plot the HMB as yearly long-term spectrograms

For memory reasons, plotting 1-minute resolution spectrograms for a full year is not possible. Therefore, we need to downsample the data (by taking the median, because values are in db). We can downsample it to hours or days.

In [ ]:
station_to_plot = 'NRS11' # By changing these value all the plots below will be for the selected stations. Feel free to change this name to any of the loaded stations

First let's look at how the full dataset looks like (not resampled)

In [ ]:
ds_collection[station_to_plot]

<div class="alert alert-block alert-info">
⚠️ Be patient, resampling a full year can take a little while...
</div>

In [ ]:
# Need to resample first, otherwise there is too much data to plot!
ds_resampled = ds_collection[station_to_plot].resample(time=RESAMPLE_RESOLUTION).median()

And this is the resampled dataset!

In [ ]:
ds_resampled

We can plot the LTSA also for an entire year

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
pypam.plots.plot_ltsa(ds=ds_resampled, data_var='psd', log=True, ax=ax, show=False, freq_coord='frequency',
                      time_coord='time')
ax.set_title(station_to_plot)
plt.show()

And if it is available, we can plot a mask with the quality flag. 
First we will resample the quality flag so any hour has the flag of its worst minute

In [ ]:
minimum_quality = 2
ds_mask = ds_collection[station_to_plot].quality_flag.resample(time=RESAMPLE_RESOLUTION).max()
ds_mask['psd'] = ds_resampled.psd
ds_mask['psd'] = ds_mask.psd.where(ds_mask < minimum_quality)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
pypam.plots.plot_ltsa(ds=ds_mask, data_var='psd', log=True, ax=ax, show=False, freq_coord='frequency',
                      time_coord='time')
ax.set_title(station_to_plot)
ax.set_facecolor('black') # -> allows us to see where the quality flag has been applied
plt.show()

### Plot the SPD

In [ ]:
# First, convert the HMB output to SPD like we did for the small wav files examples
percentiles = [1, 10, 50, 90, 99]
min_psd = 35 # in db
max_psd = 120 # in db
h = 1 # in db
ds_spd = pypam.utils.compute_spd(ds_resampled, data_var='psd', percentiles=percentiles, min_val=min_psd, 
                                 max_val=max_psd, h=h)

In [ ]:
# Plot the obtained output
fig, ax = plt.subplots(figsize=(10,5))
pypam.plots.plot_spd(spd=ds_spd, log=True, save_path=None, ax=ax, show=False)
plt.show()

### Plot a summary

We can also make the same summary plot for the entire year, not only for one day (but then we don't show day/night as it would be too busy)

In [ ]:
pypam.plots.plot_summary_dataset(ds_resampled, percentiles, data_var='psd', time_coord='time', freq_coord='frequency',
                                 min_val=min_psd, max_val=max_psd, show=True, log=True, save_path=None,
                                 location=None)

## 4. Whale presence indices

Indices are the ratio of mean power spectral density (PSD) within the frequency band of peak call energy to mean PSD at background frequencies.
These peak and background frequencies will be illustrated in the summary plot and call index calculations below

In [ ]:
# Define the frequencies that are used to compute call indices
bluepeak = np.array([42, 43])
finpeak = np.array([20, 21])
bluebackground = np.array([37, 50])
finbackground = np.array([12, 34])

We can plot a LTSA with a line showing these values, but we will resample it first to daily spectra to get daily indices. 
Be patient, resampling a full year can take a little while...

In [ ]:
whale_station1 = 'MARS' # Choose the station you want to plot
daily_resampled1 = ds_collection[whale_station1].resample(time='1D').median()  # Resample to daily spectra

In [ ]:
whale_station2 = 'NRS11' # Choose the second station you want to plot
daily_resampled2 = ds_collection[whale_station2].resample(time='1D').median()  # Resample to daily spectra

In [ ]:
# Then, plot the LTSA
fig, ax = plt.subplots(figsize=(10, 5))
pypam.plots.plot_ltsa(ds=daily_resampled1, data_var='psd', log=True, ax=ax, show=False, freq_coord='frequency',
                      time_coord='time')
ax.axhline(np.mean(bluepeak),color='black',linestyle='--',linewidth=0.5)
ax.axhline(np.mean(finpeak),color='black',linestyle='--',linewidth=0.5)
ax.set_title('NRS11')
plt.show()

In [ ]:
def compute_whale_index(peak, background, daily_ds):
    pk = daily_ds.sel(frequency=peak).psd.mean(dim='frequency')
    bg = daily_ds.sel(frequency=background).psd.mean(dim='frequency')
    CI = pk / bg
    return CI

In [ ]:
# Compute the whale indices
finCI1 = compute_whale_index(finpeak, finbackground, daily_resampled1)
blueCI1 = compute_whale_index(bluepeak, bluebackground, daily_resampled1)
finCI2 = compute_whale_index(finpeak, finbackground, daily_resampled2)
blueCI2 = compute_whale_index(bluepeak, bluebackground, daily_resampled2)

In [ ]:
# Plot the whale indices
fig, ax = plt.subplots(2, 1, figsize=(10,6), sharex=True, sharey=True)
ax[0].plot(daily_resampled1.time.values, blueCI1, '.', label='blue whale')
ax[0].plot(daily_resampled1.time.values, finCI1, 'o', markerfacecolor='none', label='fin whale')
ax[0].axhline(1, color='black', linestyle='--', linewidth=0.5)
ax[0].set_facecolor('white')
ax[0].set_title(whale_station1)

ax[1].plot(daily_resampled2.time.values, blueCI2, '.', label='blue whale')
ax[1].plot(daily_resampled2.time.values, finCI2, 'o', markerfacecolor='none', label='fin whale')
ax[1].axhline(1, color='black', linestyle='--', linewidth=0.5)
ax[1].set_facecolor('white')
ax[1].set_title(whale_station2)

plt.suptitle('Daily whale call index')
plt.ylabel('Call index')
plt.legend()
plt.show()

## 5. Daily patterns

We can plot the hourly data we have obtained for the entire year to see if there are any daily patterns which repeat

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ds_freq = pypam.utils.freq_band_aggregation(ds_resampled, data_var='psd', aggregation_freq_band=[10, 20])
pypam.plots.plot_daily_patterns_from_ds(ds=ds_freq, data_var='psd', show=True, datetime_coord='time', ax=ax)

## 6. Compare data between stations

We can compare the spectrums of two locations, by visualizing the median and the [10, 90] percentile spectrum of each location.
We first need to resample it to hourly median resolution for faster plotting. 

<div class="alert alert-block alert-info">
⚠️ Be patient, resampling a two stations can take even a longer while...
</div>

In [ ]:
# Spectral Density comparison
ds_collection_resampled = {'NRS11': ds_collection['NRS11'].resample(time='1h').median(), 
                           'MARS': ds_collection['MARS'].resample(time='1h').median()}

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax = pypam.plots.plot_multiple_spectrum_median(ds_collection_resampled, 'psd', percentiles='default', 
                                               frequency_coord='frequency',
                                               log=True, save_path=None, 
                                               ax=ax, show=True, time_coord='time')

### Aggregations and boxplots

We can now plot the evolution of all the stations by calling the bin aggregation function. It will compute the median of all the values included in the frequency band specified.
Then it will aggregate all the values of the dataset in time to match the specified time frequency. 
PyPAM has a function to plot the evolution of one or multiple stations. 

In [ ]:
aggregation_time = 'M'  # h, D, W, or M (hour, day, week or month)
aggregation_freq_band = [10, 200]
mode = 'violin'  # It can be boxplot, violin or quantiles
fig, ax = plt.subplots(figsize=(10, 6))
ax = pypam.plots.plot_multiple_aggregation_evolution(ds_dict=ds_collection,
                                                     data_var='psd',
                                                     mode=mode,
                                                     show=True,
                                                     datetime_coord='time',
                                                     aggregation_time=aggregation_time,
                                                     freq_coord='frequency',
                                                     aggregation_freq_band=aggregation_freq_band,
                                                     ax=ax,
                                                     alpha=0.5)
ax.set_title('Median sound intensity in water from frequency band %s to %s Hz' %
             (aggregation_freq_band[0], aggregation_freq_band[1]))
plt.tight_layout()

## 7. Conversions: from HMB to decidecade bands

We will work only with one day of data, just to show it (it will be faster!)

In [ ]:
ds = xr.open_dataset(local_path + '/MARS/MARS_20210316.nc')

# Convert back to upa for the sum operations
ds_decidecades = pypam.utils.hmb_to_decidecade(ds, 'psd', freq_coord='frequency', fs=256000)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(ds_decidecades.frequency, ds_decidecades['psd'].isel(time=0), label='Decidecades')
ax.plot(ds.frequency, ds['psd'].isel(time=0), label='HMB')
plt.xscale('symlog')
plt.xlim(left=10)
ax.set_facecolor('white')
plt.legend()
plt.show()